# Predicting Baseball Salaries, Part II

I previously worked on a simple regression model to predict baseball player salaries where I collected player team, position and salary for the 2019 season. It was a simple exercise in using decision trees and different ensemble methods to predict salaries, but I wanted to go back adn see if I included more stats, specifically the previous season's 

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
salary = pd.read_pickle('salary_info')

In [4]:
salary.head()

,name,team,position,salary
0,MaxScherzer,WSH,SP,42142857
1,StephenStrasburg,WSH,SP,36428571
2,MikeTrout,LAA,CF,34083333
3,ZackGreinke,ARI,SP,32421884
4,DavidPrice,BOS,SP,31000000
